In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import plotly.offline as pl
pl.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.ensemble import RandomForestClassifier


import parseIntervalFiles as pif
import consolidateFiles as cf
import datacleaning as cl
import dataviz as dv
import fragmentation as fr
import classif_multiclass as cmc
import hervpd as hp

## Pipeline configuration 
* set the input/output directories, users and date range

In [2]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"

dt1 = datetime(2018, 6, 18)
dt2 = datetime(2018, 6, 25)

users = [0, 1]

## Extracting sessions

In [3]:
%time sess = cf.gen_sessions_dataset(users, dt1, dt2, dirname=RAW_PATH, verbose=False)

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 97, using nperseg = 97



Wall time: 3.29 s


In [4]:
dfs = pd.DataFrame(sess)

In [5]:
dfs = dfs[dfs.activity.isin(['train-baseline', 'train-focus', 'train-breathe'])]

In [6]:
dfs.groupby('activity').count()['duration']

activity
train-baseline    13
train-breathe     18
train-focus       18
Name: duration, dtype: int64

In [66]:
dfs[dfs.user==0][['activity', 'user', 'start', 'duration', 'beatscount']]

,activity,user,start,duration,beatscount
0,train-baseline,0,2018-06-18 11:02:20,128,154
1,train-focus,0,2018-06-18 11:04:32,128,165
2,train-breathe,0,2018-06-18 11:10:18,131,168
3,train-focus,0,2018-06-18 11:14:17,129,169
4,train-breathe,0,2018-06-18 11:24:58,128,165
5,train-focus,0,2018-06-18 12:35:13,134,191
6,train-breathe,0,2018-06-18 12:39:50,133,190
7,train-baseline,0,2018-06-18 17:22:39,153,204
8,train-focus,0,2018-06-18 17:25:34,135,179
9,train-breathe,0,2018-06-18 17:29:15,135,175


In [8]:
dfs.describe()

,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,removed_artifacts,rmssd,sdnn,sess_id,total_power,user,vlf
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,177.102041,133.020408,950.315925,32.865064,2299.550537,5.171729,67.134936,79.722853,768.742749,24.469388,14.391257,0.673469,53.536492,71.057760,25.816327,3987.728261,0.306122,737.861799
std,19.830789,9.512031,1358.916364,23.972474,2982.618442,5.436716,23.972474,7.699467,82.521754,18.203825,11.114657,1.712549,32.786063,29.019607,15.743614,3790.150844,0.465657,850.013169
min,132.000000,104.000000,36.467514,4.565933,130.060979,0.217607,17.871682,60.426939,664.222826,0.000000,0.000000,0.000000,12.977209,24.809968,0.000000,354.178357,0.000000,70.989653
25%,165.000000,128.000000,213.821800,11.179485,503.357147,0.772207,43.573176,76.333517,705.962766,10.000000,5.970149,0.000000,31.108347,51.481478,12.000000,1197.478684,0.000000,264.773195
50%,182.000000,133.000000,405.797394,26.423060,1309.118840,2.784573,73.576940,80.603377,749.734807,23.000000,12.234043,0.000000,52.527403,65.083679,27.000000,2260.807748,0.000000,466.142752
75%,190.000000,136.000000,932.902562,56.426824,2469.443406,7.944956,88.820515,85.371274,794.587879,37.000000,19.211823,1.000000,63.384904,92.249014,39.000000,5351.542867,1.000000,634.662141
max,212.000000,171.000000,6015.982540,82.128318,14893.856347,20.901331,95.434067,92.160516,1006.560606,68.000000,41.975309,10.000000,153.353970,141.658789,52.000000,16808.251518,1.000000,3940.988059


In [9]:
dfs.drop(['rr'], axis =  1).to_excel(PRE_PATH + "\\train_sessions.xlsx")


---


# VISUALIZATION

---



In [10]:
#d21 = df_export[df_export.start > '2018-06-21']
for feat in cl.features_all:
    dv.boxplot_compare(dfs, feat, groupby='activity', min_examples=1)


---


# FRAGMENTATION

---



In [20]:
frags = fr.gen_fragments_dataset(sessions, 60, 10, RAW_PATH)

49 valid sessions out of 49 total (at least one full fragment of 60 seconds after discarding first 10 seconds)


C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 237, using nperseg = 237

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 239, using nperseg = 239

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 241, using nperseg = 241

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 240, using nperseg = 240

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 238, using nperseg = 238

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 245, using nperseg = 245

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spect

In [22]:
dff = pd.DataFrame(frags)

In [23]:
dff.sample(5)

,activity,beatcount,f_id,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,rmssd,rr,sdnn,sess_id,start,stop,total_power,user,vlf
57,train-breathe,85,0,248.472642,8.341820,2730.165475,10.987791,91.658180,85.985055,712.529412,...,sit,66.321547,"[{'date': 2018-06-21 11:15:23, 'interval': 720...",88.322675,36,2018-06-21 11:15:22,2018-06-21 11:16:22,3150.469791,0,171.831675
47,train-breathe,82,1,192.582796,8.870920,1978.362334,10.272789,91.129080,81.865729,736.121951,...,sit,25.683015,"[{'date': 2018-06-20 22:43:21, 'interval': 750...",48.818229,29,2018-06-20 22:43:20,2018-06-20 22:44:20,2209.467946,0,38.522815
35,train-breathe,84,0,309.832069,10.385385,2673.514846,8.628916,89.614615,84.752596,711.488095,...,sit,29.065775,"[{'date': 2018-06-19 13:55:11, 'interval': 696...",51.274009,21,2018-06-19 13:55:10,2018-06-19 13:56:10,3193.329136,0,209.982221
39,train-baseline,82,0,174.556629,36.682961,301.295435,1.726061,63.317039,83.777754,722.231707,...,sit,84.074188,"[{'date': 2018-06-20 01:54:30, 'interval': 138...",84.028258,25,2018-06-20 01:54:28,2018-06-20 01:55:28,833.559864,0,357.707800
7,train-focus,87,1,229.887232,27.941240,592.865195,2.578939,72.058760,84.681671,710.034483,...,sit,27.462236,"[{'date': 2018-06-18 12:36:24, 'interval': 700...",32.262170,5,2018-06-18 12:36:23,2018-06-18 12:37:23,1028.258437,0,205.506010


In [27]:
durations = [60, 30, 20, 10]
crop = 10

for dr in durations:
    fname = PRE_PATH + '\\df_train_' + str(dr) + '.xlsx'
    print ('generating', fname, '...')
    ds = pd.DataFrame(fr.gen_fragments_dataset(sessions, dr, crop, RAW_PATH))
    print('resulting dataset:', len(ds), 'records')
    ds.drop('rr', axis=1).to_excel(fname)


---


# CLASSIFICATION

---



In [54]:
features = cl.features_fd
clf1 = RandomForestClassifier(n_estimators=10, n_jobs=-1)
clf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1)
clf3 = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf4 = RandomForestClassifier(n_estimators=10000, n_jobs=-1)
clf5 = RandomForestClassifier(n_estimators=100000, n_jobs=-1)

In [57]:
dff = pd.read_excel(PRE_PATH+'\\df_train_60.xlsx')
dff.describe()

,beatcount,f_id,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,rmssd,sdnn,sess_id,total_power,user,vlf
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,78.547619,0.416667,760.255933,35.404996,1992.785213,4.891232,64.595004,78.859941,776.504373,10.369048,14.004936,46.698393,64.429426,25.928571,3522.883017,0.297619,769.841871
std,8.074062,0.495968,1320.252267,24.445693,3023.320605,6.588052,24.445693,7.914989,87.561033,8.682873,12.198991,32.756000,33.527025,15.626503,4061.127369,0.459957,1060.614932
min,60.000000,0.000000,5.133648,2.425866,34.110708,0.153289,13.291445,59.485874,653.336957,0.000000,0.000000,10.034424,14.327046,0.000000,62.899132,0.000000,22.317737
25%,75.000000,0.000000,202.761845,12.320195,387.518461,0.847233,45.860254,75.968714,708.491176,3.000000,4.425872,24.824779,42.101630,12.000000,1037.381298,0.000000,166.233359
50%,80.000000,0.000000,315.957325,31.871658,737.886536,2.143273,68.128342,79.430434,758.266526,9.000000,11.476608,37.114574,53.033829,26.000000,1931.934026,0.000000,336.117780
75%,85.000000,1.000000,879.140438,54.139746,2233.993247,7.120456,87.679805,85.271553,801.006053,15.250000,20.263158,57.326750,84.098196,39.250000,3981.717115,1.000000,774.382533
max,92.000000,1.000000,9555.447872,86.708555,14522.536141,40.222397,97.574134,91.999542,1019.600000,42.000000,59.154930,198.731328,182.668902,52.000000,18179.696118,1.000000,5548.117512


In [58]:
dfr = dff[dff.activity.isin(['train-focus', 'train-breathe'])]

In [63]:
for label in ['activity']:
    cmc.plot_matrices(dfr, clf4, features, label)




--------------------------------------------- activity ---------------------------------------------
activity
train-breathe    32
train-focus      30
Name: user, dtype: int64
49 13
